In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls /content/drive/My\ Drive/Colab\ Notebooks

 558_Assignments_Code		  Kaggle
 6.1.2_medical_transcript.ipynb   my_own_code
 data				 'powerful word.ipynb'
'Fondation Final Project'


In [0]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import pandas as pd
from gensim.models import KeyedVectors
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
mt = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fondation Final Project/data/medical data/medicaltranscriptions.csv')
icd = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Fondation Final Project/data/medical data/allvalid2011 (detailed titles headings).xlsx')

In [0]:
mt

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."
...,...,...,...,...,...,...
4994,4994,Patient having severe sinusitis about two to ...,Allergy / Immunology,Chronic Sinusitis,"HISTORY:, I had the pleasure of meeting and e...",NaN
4995,4995,This is a 14-month-old baby boy Caucasian who...,Allergy / Immunology,Kawasaki Disease - Discharge Summary,"ADMITTING DIAGNOSIS: , Kawasaki disease.,DISCH...","allergy / immunology, mucous membranes, conjun..."
4996,4996,A female for a complete physical and follow u...,Allergy / Immunology,Followup on Asthma,"SUBJECTIVE: , This is a 42-year-old white fema...",NaN
4997,4997,Mother states he has been wheezing and coughing.,Allergy / Immunology,Asthma in a 5-year-old,"CHIEF COMPLAINT: , This 5-year-old male presen...",NaN


In [0]:
# word2vec pretrained model
model = KeyedVectors.load_word2vec_format("/content/drive/My Drive/Colab Notebooks/Fondation Final Project/data/medical data/PubMed-and-PMC-w2v.bin", binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# medical transcript

In [0]:
mt = mt.fillna('')
mt['vec_str'] = mt['description']+mt['medical_specialty']+mt['keywords']

In [0]:
mt

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,vec_str
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller...",A 23-year-old white female presents with comp...
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh...",Consult for laparoscopic gastric bypass. Bari...
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart...",Consult for laparoscopic gastric bypass. Bari...
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple...",2-D M-Mode. Doppler. Cardiovascular / Pulmo...
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo...",2-D Echocardiogram Cardiovascular / Pulmonary...
...,...,...,...,...,...,...,...
4994,4994,Patient having severe sinusitis about two to ...,Allergy / Immunology,Chronic Sinusitis,"HISTORY:, I had the pleasure of meeting and e...",,Patient having severe sinusitis about two to ...
4995,4995,This is a 14-month-old baby boy Caucasian who...,Allergy / Immunology,Kawasaki Disease - Discharge Summary,"ADMITTING DIAGNOSIS: , Kawasaki disease.,DISCH...","allergy / immunology, mucous membranes, conjun...",This is a 14-month-old baby boy Caucasian who...
4996,4996,A female for a complete physical and follow u...,Allergy / Immunology,Followup on Asthma,"SUBJECTIVE: , This is a 42-year-old white fema...",,A female for a complete physical and follow u...
4997,4997,Mother states he has been wheezing and coughing.,Allergy / Immunology,Asthma in a 5-year-old,"CHIEF COMPLAINT: , This 5-year-old male presen...",,Mother states he has been wheezing and coughi...


In [0]:
# Generate Vec for each row
description_vec_mt=np.empty((0, 200), float)

stop_words = set(stopwords.words('english'))
for i in range(len(mt['description'])):
    word2vec_ls = np.empty((0,200),dtype=float)
    #tokenize each sentence & remove stop words
    word_tokens = nltk.word_tokenize(mt['vec_str'][i])
    word_tokens = [w.lower() for w in word_tokens] 
    word_tokens = [w.lower() for w in word_tokens if not w in stop_words] 
    word_tokens = [w.lower() for w in word_tokens if w.isalpha()]
    #pos tag & remove useless words & document representation
    mt_token_pos = nltk.pos_tag(word_tokens)
    for word, pos in mt_token_pos:
        if (pos=='NN' or pos=='NNP' or pos=='NNS' or pos=='NNPS'):
            try:
                word2vec_ls = np.vstack([word2vec_ls, model.word_vec(word)])
            except:
                pass
    if np.any(np.isnan(word2vec_ls)) or len(word2vec_ls)==0:
        desc_vec = np.zeros((1,200))
    else:
        desc_vec = np.mean(word2vec_ls, axis=0).reshape(1,200)
    description_vec_mt = np.vstack([description_vec_mt, desc_vec])

In [0]:
description_vec_mt

array([[-0.16527502,  0.23292068,  0.03101953, ..., -0.11836763,
        -0.15297651, -0.12170596],
       [-0.0765232 ,  0.07079678,  0.00120748, ...,  0.0217549 ,
        -0.03041719, -0.05852601],
       [-0.13290381,  0.13185818,  0.05476694, ..., -0.01014871,
        -0.0279903 , -0.10029698],
       ...,
       [-0.14297709,  0.30390154, -0.03104285, ..., -0.0348169 ,
        -0.3089878 , -0.19330167],
       [-0.14509307,  0.02270136, -0.07112908, ...,  0.18559933,
        -0.14628446, -0.1484046 ],
       [-0.15438843,  0.20730674, -0.01857861, ...,  0.13657174,
        -0.13328435, -0.12162492]])

# ICD Title

In [0]:
icd

,Status,Code,ICD Title
0,NaN,A00-B99,I. Certain infectious and parasitic diseases
1,NaN,A00-A09,Intestinal infectious diseases
2,NaN,A00,Cholera
3,NaN,A00.0,"Cholera due to Vibrio cholerae 01, biovar chol..."
4,NaN,A00.1,"Cholera due to Vibrio cholerae 01, biovar eltor"
...,...,...,...
11123,NaN,Y88.3,Sequelae of surgical and medical procedures as...
11124,NaN,Y89,Sequelae of other external causes
11125,NaN,Y89.0,Sequelae of legal intervention
11126,NaN,Y89.1,Sequelae of war operations


In [0]:
#
import collections
code_title_dic = collections.defaultdict(list)
icd_titles = icd['ICD Title'].values.tolist()
icd_codes = icd['Code'].values.tolist()

main_code = None

for code,des in zip(icd_codes,icd_titles):
    if '-' in code:
        continue
    elif '.' in code and main_code in code:
        code_title_dic[main_code].append(des)
    else:
        main_code = code
        code_title_dic[main_code].append(des)

In [0]:
code_title_dic['A02']

['Other salmonella infections',
 'Salmonella enteritis',
 'Salmonella septicemia',
 'Localized salmonella infections',
 'Other specified salmonella infections',
 'Salmonella infection, unspecified']

In [0]:
for k, v in code_title_dic.items():
  s = ''
  for each in code_title_dic[k]:
    s+=each+' '
  word_tokens = nltk.word_tokenize(s)
  word_tokens = [w.lower() for w in word_tokens] 
  word_tokens = [w.lower() for w in word_tokens if not w in stop_words] 
  word_tokens = [w.lower() for w in word_tokens if w.isalpha()]
  word_tokens = list(set(word_tokens))
  mt_token_pos = nltk.pos_tag(word_tokens)
  word_tokens=[]
  for word, pos in mt_token_pos:
      if (pos=='NN' or pos=='NNP' or pos=='NNS' or pos=='NNPS'):
        word_tokens.append(word)
  code_title_dic[k] = word_tokens

In [0]:
'''s = ''
for each in code_title_dic['A02']:
  s+=each+' '
word_tokens = nltk.word_tokenize(s)
word_tokens = [w.lower() for w in word_tokens] 
word_tokens = [w.lower() for w in word_tokens if not w in stop_words] 
word_tokens = [w.lower() for w in word_tokens if w.isalpha()]
word_tokens = list(set(word_tokens))
mt_token_pos = nltk.pos_tag(word_tokens)
word_tokens=[]
for word, pos in mt_token_pos:
    if (pos=='NN' or pos=='NNP' or pos=='NNS' or pos=='NNPS'):
      word_tokens.append(word)'''

"s = ''\nfor each in code_title_dic['A02']:\n  s+=each+' '\nword_tokens = nltk.word_tokenize(s)\nword_tokens = [w.lower() for w in word_tokens] \nword_tokens = [w.lower() for w in word_tokens if not w in stop_words] \nword_tokens = [w.lower() for w in word_tokens if w.isalpha()]\nword_tokens = list(set(word_tokens))\nmt_token_pos = nltk.pos_tag(word_tokens)\nword_tokens=[]\nfor word, pos in mt_token_pos:\n    if (pos=='NN' or pos=='NNP' or pos=='NNS' or pos=='NNPS'):\n      word_tokens.append(word)"

In [0]:
code_title_dic['A02']

['infections', 'septicemia', 'enteritis', 'infection', 'salmonella']

In [0]:
#generate vector for each medical case
description_vec_icd = np.empty((0, 200), float)
for k,v in code_title_dic.items():
  for word in code_title_dic[k]:
    word2vec_ls = np.empty((0,200),dtype=float)
    try:
      word2vec_ls = np.vstack([word2vec_ls, model.word_vec(word)])
    except:
      pass
  if np.any(np.isnan(word2vec_ls)) or len(word2vec_ls)==0:
      desc_vec = np.zeros((1,200))
  else:
      desc_vec = np.mean(word2vec_ls, axis=0).reshape(1,200)
  description_vec_icd = np.vstack([description_vec_icd, desc_vec])

In [0]:
len(code_title_dic)

1855

In [0]:
len(description_vec_icd)

1855

In [0]:
description_vec_icd

array([[ 0.04254275, -0.23826714,  0.28815874, ..., -0.26780584,
        -0.12462866, -0.51430589],
       [ 0.11146717,  0.63124722, -0.1294912 , ...,  0.21747549,
         0.15763395, -0.39637721],
       [-0.11221837,  0.50359046,  0.14234984, ...,  0.32154804,
        -0.13267078, -0.02983945],
       ...,
       [-0.34983236,  0.11156194, -0.17262951, ...,  0.18101849,
        -0.27067941, -0.07533946],
       [ 0.09030204, -0.00942799,  0.15827665, ...,  0.207193  ,
         0.20037232, -0.29499066],
       [ 0.10345531,  0.02581475, -0.04503417, ...,  0.42783907,
        -0.27521664,  0.00408921]])

In [0]:
code_vec_dic = collections.defaultdict(list)
icd_codes = list(code_title_dic.keys())
icd_vec = description_vec_icd

main_code = None

for code,des in zip(icd_codes, icd_vec):
  main_code = code
  code_vec_dic[main_code].append(des)

In [0]:
code_vec_dic['A01']

[array([ 1.11467168e-01,  6.31247222e-01, -1.29491195e-01, -3.07447821e-01,
        -1.05877720e-01, -1.29759284e-02, -5.11511974e-02,  1.26132250e-01,
        -2.75561102e-02,  5.57404757e-01,  3.09111178e-01, -4.56625521e-01,
         7.33596087e-02, -2.43103579e-01, -1.37701347e-01, -1.74123496e-01,
         7.23797083e-02, -1.72110617e-01,  1.60197243e-01, -1.12725981e-01,
         4.23627824e-01, -3.65239056e-03, -1.19522773e-01,  1.88164458e-01,
         4.13232058e-01,  1.54701769e-01,  1.87907800e-01,  2.23215505e-01,
        -3.49307120e-01, -1.89084396e-01,  6.48076162e-02,  4.21708636e-02,
        -1.42581221e-02, -2.16550037e-01, -8.06223880e-03,  2.61910439e-01,
         1.20327234e-01, -2.35844687e-01, -2.20286384e-01, -3.76981159e-04,
         1.67585567e-01, -1.73180446e-01, -5.87958872e-01, -1.77297533e-01,
        -1.07906096e-01, -9.92268473e-02, -3.99029493e-01, -1.75644740e-01,
        -2.44319156e-01,  7.62623176e-02, -2.92000502e-01,  5.36587536e-02,
        -1.7

# compare similarity

In [0]:
len(code_vec_dic)

1855

In [0]:
# code_vec_dic.values represents icd vector representation
# description_vec_mt represents medical transcript vector representation

#build similarity matrix.
similarity_matrix = pd.DataFrame(np.zeros((4999, len(code_vec_dic)), dtype=float), columns=icd_codes)
similarity_matrix

,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A30,A31,A32,A33,A34,A35,A36,A37,A38,A39,A40,A41,A42,A43,A44,A46,A48,...,Y48,Y49,Y50,Y51,Y52,Y53,Y54,Y55,Y56,Y57,Y58,Y59,Y60,Y61,Y62,Y63,Y64,Y65,Y66,Y69,Y70,Y71,Y72,Y73,Y74,Y75,Y76,Y77,Y78,Y79,Y80,Y81,Y82,Y83,Y84,Y85,Y86,Y87,Y88,Y89
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(4999):
  print('Calculating {}th row'.format(i+1))
  for j in similarity_matrix.columns:
    similarity_matrix.loc[i,j] = cosine_similarity(description_vec_mt[i].reshape(1,-1), code_vec_dic[j][0].reshape(1,-1))

Calculating 1th row
Calculating 2th row
Calculating 3th row
Calculating 4th row
Calculating 5th row
Calculating 6th row
Calculating 7th row
Calculating 8th row
Calculating 9th row
Calculating 10th row
Calculating 11th row
Calculating 12th row
Calculating 13th row
Calculating 14th row
Calculating 15th row
Calculating 16th row
Calculating 17th row
Calculating 18th row
Calculating 19th row
Calculating 20th row
Calculating 21th row
Calculating 22th row
Calculating 23th row
Calculating 24th row
Calculating 25th row
Calculating 26th row
Calculating 27th row
Calculating 28th row
Calculating 29th row
Calculating 30th row
Calculating 31th row
Calculating 32th row
Calculating 33th row
Calculating 34th row
Calculating 35th row
Calculating 36th row
Calculating 37th row
Calculating 38th row
Calculating 39th row
Calculating 40th row
Calculating 41th row
Calculating 42th row
Calculating 43th row
Calculating 44th row
Calculating 45th row
Calculating 46th row
Calculating 47th row
Calculating 48th row
C

In [0]:
similarity_matrix

,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A30,A31,A32,A33,A34,A35,A36,A37,A38,A39,A40,A41,A42,A43,A44,A46,A48,...,Y50,Y51,Y52,Y53,Y54,Y55,Y56,Y57,Y58,Y59,Y60,Y61,Y62,Y63,Y64,Y65,Y66,Y69,Y70,Y71,Y72,Y73,Y74,Y75,Y76,Y77,Y78,Y79,Y80,Y81,Y82,Y83,Y84,Y85,Y86,Y87,Y88,Y89,max_index,max_prob
0,0.026074,0.448598,0.215652,0.169547,0.085681,-0.034165,0.205466,0.203477,0.360185,0.453587,0.077050,0.182978,0.377504,0.252529,0.128231,0.369948,0.127808,0.098059,0.252529,0.433509,0.131187,0.130110,0.274488,0.203477,0.205466,0.127808,0.178459,0.178459,0.178459,0.142052,0.136957,0.448598,0.252529,-0.028994,0.050428,0.306242,0.127808,0.127808,0.451563,0.0,...,0.186743,0.209723,0.091409,0.360302,0.123843,0.054124,0.331416,0.030469,0.099248,0.058531,-0.040710,-0.040710,-0.040710,0.199004,0.220156,0.027234,0.162231,0.153140,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.130959,0.199996,0.173952,-0.040710,0.271380,0.294085,0.064202,0.173952,0.180391,J30,0.754661
1,0.019456,0.151921,0.013340,-0.012869,0.039627,-0.002846,0.142382,0.147403,0.049392,0.239336,0.068022,0.072123,0.060074,0.128047,0.026142,-0.075832,0.148826,-0.022669,0.128047,0.123034,-0.062104,0.004337,-0.005808,0.147403,0.142382,0.148826,0.049344,0.049344,0.049344,-0.049913,0.064610,0.151921,0.128047,0.039291,-0.033113,0.044011,0.148826,0.148826,0.012578,0.0,...,-0.052672,0.060649,0.008961,0.231358,-0.059652,0.122590,-0.004250,-0.011132,0.078985,0.017546,0.122950,0.122950,0.122950,0.164966,0.061099,0.059107,0.090576,0.236482,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.228241,0.176164,0.202111,0.122950,0.102914,0.079242,0.195165,0.202111,0.133385,D50,0.548074
2,-0.014939,0.163079,0.045342,0.044433,0.017048,0.016677,0.150356,0.241952,0.137304,0.224940,0.143417,0.077072,0.070516,0.268778,0.123623,-0.026696,0.041743,-0.044580,0.268778,0.123922,-0.076073,0.088760,0.049214,0.241952,0.150356,0.041743,0.039990,0.039990,0.039990,-0.048133,0.000101,0.163079,0.268778,0.036089,-0.034964,0.177737,0.041743,0.041743,0.092559,0.0,...,-0.007619,0.145603,0.006860,0.179582,-0.006890,0.101605,-0.009698,-0.037727,-0.000692,-0.011931,0.215078,0.215078,0.215078,0.175787,0.038003,0.070912,0.117934,0.230076,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.247539,0.202966,0.232664,0.215078,0.194271,0.170870,0.248808,0.232664,0.172687,H59,0.662815
3,0.026513,0.135662,0.039851,0.003296,0.032318,0.082930,0.101269,0.167935,0.175855,0.137657,0.110944,0.091970,0.083633,0.184592,0.095259,0.021598,0.042892,0.079491,0.184592,0.069730,-0.029300,-0.007063,0.060740,0.167935,0.101269,0.042892,0.021652,0.021652,0.021652,0.087548,-0.028894,0.135662,0.184592,0.001706,0.012830,0.172901,0.042892,0.042892,0.041767,0.0,...,-0.047537,0.024533,0.043092,0.030223,0.058737,0.127166,-0.029659,0.018877,-0.006526,0.078879,0.192056,0.192056,0.192056,0.080938,-0.034227,0.227200,0.096190,0.004380,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.112916,0.072081,0.091467,0.192056,0.101694,0.046089,0.081032,0.091467,-0.001974,I50,0.671866
4,0.011046,0.108014,-0.001801,-0.040168,0.008490,0.026372,0.095513,0.152703,0.130312,0.108499,0.141786,0.066100,0.145210,0.169432,0.047066,-0.015027,0.064869,0.032353,0.169432,0.032554,-0.051156,0.004553,0.042336,0.152703,0.095513,0.064869,0.008141,0.008141,0.008141,0.026661,-0.024269,0.108014,0.169432,-0.005614,-0.037363,0.221342,0.064869,0.064869,0.052019,0.0,...,-0.038899,0.085043,0.061453,0.064518,0.038458,0.201387,0.010800,0.040443,-0.005056,0.015204,0.167040,0.167040,0.167040,0.088708,0.001563,0.256101,0.064527,0.021454,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.092948,0.113825,0.097463,0.167040,0.134355,0.035131,0.085959,0.097463,0.004841,I50,0.683359
...,...,...,...,...,...,...,...,...,.

In [0]:
max_index=[]
max_val = []
for i in range(4999):
  max_index.append(np.argmax(similarity_matrix.iloc[i], axis=1))
  max_val.append(similarity_matrix.loc[i,max_index[i]])

In [0]:
similarity_matrix['max_index'] = max_index

In [0]:
similarity_matrix['max_prob'] = max_val

In [0]:
similarity_matrix.to_csv('/content/drive/My Drive/Colab Notebooks/Fondation Final Project/data/medical data/similarity_matrix_2.csv')

In [0]:
similarity_matrix

,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A30,A31,A32,A33,A34,A35,A36,A37,A38,A39,A40,A41,A42,A43,A44,A46,A48,...,Y50,Y51,Y52,Y53,Y54,Y55,Y56,Y57,Y58,Y59,Y60,Y61,Y62,Y63,Y64,Y65,Y66,Y69,Y70,Y71,Y72,Y73,Y74,Y75,Y76,Y77,Y78,Y79,Y80,Y81,Y82,Y83,Y84,Y85,Y86,Y87,Y88,Y89,max_index,max_prob
0,0.026074,0.448598,0.215652,0.169547,0.085681,-0.034165,0.205466,0.203477,0.360185,0.453587,0.077050,0.182978,0.377504,0.252529,0.128231,0.369948,0.127808,0.098059,0.252529,0.433509,0.131187,0.130110,0.274488,0.203477,0.205466,0.127808,0.178459,0.178459,0.178459,0.142052,0.136957,0.448598,0.252529,-0.028994,0.050428,0.306242,0.127808,0.127808,0.451563,0.0,...,0.186743,0.209723,0.091409,0.360302,0.123843,0.054124,0.331416,0.030469,0.099248,0.058531,-0.040710,-0.040710,-0.040710,0.199004,0.220156,0.027234,0.162231,0.153140,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.130959,0.199996,0.173952,-0.040710,0.271380,0.294085,0.064202,0.173952,0.180391,J30,0.754661
1,0.019456,0.151921,0.013340,-0.012869,0.039627,-0.002846,0.142382,0.147403,0.049392,0.239336,0.068022,0.072123,0.060074,0.128047,0.026142,-0.075832,0.148826,-0.022669,0.128047,0.123034,-0.062104,0.004337,-0.005808,0.147403,0.142382,0.148826,0.049344,0.049344,0.049344,-0.049913,0.064610,0.151921,0.128047,0.039291,-0.033113,0.044011,0.148826,0.148826,0.012578,0.0,...,-0.052672,0.060649,0.008961,0.231358,-0.059652,0.122590,-0.004250,-0.011132,0.078985,0.017546,0.122950,0.122950,0.122950,0.164966,0.061099,0.059107,0.090576,0.236482,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.228241,0.176164,0.202111,0.122950,0.102914,0.079242,0.195165,0.202111,0.133385,D50,0.548074
2,-0.014939,0.163079,0.045342,0.044433,0.017048,0.016677,0.150356,0.241952,0.137304,0.224940,0.143417,0.077072,0.070516,0.268778,0.123623,-0.026696,0.041743,-0.044580,0.268778,0.123922,-0.076073,0.088760,0.049214,0.241952,0.150356,0.041743,0.039990,0.039990,0.039990,-0.048133,0.000101,0.163079,0.268778,0.036089,-0.034964,0.177737,0.041743,0.041743,0.092559,0.0,...,-0.007619,0.145603,0.006860,0.179582,-0.006890,0.101605,-0.009698,-0.037727,-0.000692,-0.011931,0.215078,0.215078,0.215078,0.175787,0.038003,0.070912,0.117934,0.230076,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.247539,0.202966,0.232664,0.215078,0.194271,0.170870,0.248808,0.232664,0.172687,H59,0.662815
3,0.026513,0.135662,0.039851,0.003296,0.032318,0.082930,0.101269,0.167935,0.175855,0.137657,0.110944,0.091970,0.083633,0.184592,0.095259,0.021598,0.042892,0.079491,0.184592,0.069730,-0.029300,-0.007063,0.060740,0.167935,0.101269,0.042892,0.021652,0.021652,0.021652,0.087548,-0.028894,0.135662,0.184592,0.001706,0.012830,0.172901,0.042892,0.042892,0.041767,0.0,...,-0.047537,0.024533,0.043092,0.030223,0.058737,0.127166,-0.029659,0.018877,-0.006526,0.078879,0.192056,0.192056,0.192056,0.080938,-0.034227,0.227200,0.096190,0.004380,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.112916,0.072081,0.091467,0.192056,0.101694,0.046089,0.081032,0.091467,-0.001974,I50,0.671866
4,0.011046,0.108014,-0.001801,-0.040168,0.008490,0.026372,0.095513,0.152703,0.130312,0.108499,0.141786,0.066100,0.145210,0.169432,0.047066,-0.015027,0.064869,0.032353,0.169432,0.032554,-0.051156,0.004553,0.042336,0.152703,0.095513,0.064869,0.008141,0.008141,0.008141,0.026661,-0.024269,0.108014,0.169432,-0.005614,-0.037363,0.221342,0.064869,0.064869,0.052019,0.0,...,-0.038899,0.085043,0.061453,0.064518,0.038458,0.201387,0.010800,0.040443,-0.005056,0.015204,0.167040,0.167040,0.167040,0.088708,0.001563,0.256101,0.064527,0.021454,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.092948,0.113825,0.097463,0.167040,0.134355,0.035131,0.085959,0.097463,0.004841,I50,0.683359
...,...,...,...,...,...,...,...,...,.